In [1]:
import ctypes
#hllDll = ctypes.WinDLL("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.1\\bin\\cudart64_101.dll")
#incase cudart64_101.dll is unable to be loaded
import logging
import os
import numpy as np
from medpy.io import load
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras import Input, Model, models, layers
import keras
from keras.utils.vis_utils import plot_model
import sklearn
from scipy.ndimage import rotate
import scipy
import time
from sklearn import datasets, preprocessing, model_selection, metrics
import matplotlib.pyplot as plt
import pydot
import graphviz
import pydotplus
%matplotlib inline

#logging.basicConfig(level = 'logging.info')


In [2]:
'''Confirming running on GPU'''

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
'''
CUDA_VISIBLE_DEVICES = 0,1
'''
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7824564768173545159
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16500547467963836693
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3048629862
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13502513082923641504
physical_device_desc: "device: 0, name: GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6757887804049758187
physical_device_desc: "device: XLA_GPU device"
]
Num GPUs Available:  1


In [3]:
'''Predetermined Functions'''
def Generic_image_generator(URL):
    image_data, image_header = load(URL)
    for iter in range(0,len(image_data[:][:])):
        yield image_data[iter]

def create_ID_dict(basepath): #currently only set to return CT and segmentation mask images, not CTA's
    '''basepath is location of file that is holding dataset, datasets should be deeper in the file tree'''
    logging.info("creating dataset location directory/dictionary from {}".format(basepath))
    dict = {}
    mask_list = []
    image_list = []
    for root, dirs, list_of_files in os.walk(basepath):
        for files in list_of_files:
            if "r.mhd" in files: #checks if files ends in r.mhd, signifying a segmentation truth. appeneded to masked list
                mask_list.append(os.path.join(root,files))
            else: 
                pass
            
            if "cti.mhd" in files: #checks if files end in CTI.mhd, and appends it to image list
                image_list.append(os.path.join(root,files))
            else:
                pass
    #for the current moment CTA images are not being uploaded.
            
    dict.update({"image" : image_list })
    dict.update({"mask" : mask_list })
    logging.info("dictionary created")
    
    return dict

def image_preprocessing(array):
    logging.info("Beginning Preprocessing")
    
    logging.info("Rotating Image into axial plane")
    array = rotate(array, 90, axes=(0,2), reshape=True) #rotates to axial form sup to inf direction.
    
    logging.info("Making image to uniform size")
    
    blank_array = np.full((1,512,512),-3024) #-3024 is the "background" of the images, and is considered the value for non truth in the segmentation masks"
    
    while len(array) < 69: #this while loop with following if statement helps bring all images to size 70x512x512
        array = np.append(array, blank_array, axis = 0)
        array = np.append(blank_array, array, axis = 0)
        
    if len(array) == 69:
        array = np.append(array,blank_array, axis = 0)
    else:
        pass
    
    logging.info("Image Preprocessing is complete")
    
    return array
    
            
def image_generator(img_dict, mask):
    
    include_mask = False
    
    logging.info("creating generator for {}.".format(img_dict))
    
    if mask == True: #checking if dictoinary contains assoicated mask images.
        include_mask = True
    
    if include_mask == True:
        
        for path1, path2 in zip(img_dict['image'],img_dict['mask']):
            logging.info("loading {} {} into generator".format("image",os.path.basename(path1)))
            image, header_img = load(path1) #load simage, and gets image header. Currently image header is not being exported from generator.
            print("Image processing")
            image = image_preprocessing(image)
            
            
            logging.info("loading {} {} into generator".format("mask",os.path.basename(path2)))
            mask_img, header_mask = load(path2) #load mask, and gets mask image header. Currently mask header is not being exported from generator.
            print("Mask Processing")
            mask_img = image_preprocessing(mask_img)
        
            logging.info("Generator is prepped")
            
            for img_slice, mask_slice in zip(image,mask_img): #This for loop is to make image the correct tensor length for TF.
                img_slice = np.reshape(img_slice,(1,1,512,512))
                mask_slice = np.reshape(mask_slice,(1,1,512,512))
                
                yield img_slice, mask_slice
            
    elif include_mask == False:
        for path in dict_obj["image"]: #load simage, and gets image header. Currently image header is not being exported from generator.
            logging.info("loading {} {} into generator".format("image",os.path.basename(path)))
            image, header  = load(path)
            image = image_preprocessing(image)
            logging.info("Generator is prepped")
            
            for slice in image:  #This for loop is to make image the correct tensor length for TF.
                slice = np.reshape(slice,(1,1,512,512))
                yield slice
    else:
        raise TypeError("Problem with inputs.")
        logging.error("Unable to load image into generator")
            
    #no normalization has been added yet. Will need to think about how I want to approach this.
    #from skimage.color import rgb2gray to make things grayscale.

In [4]:
'''Predetermined Variables'''
PATH_tr = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Training_Set\\Train\\'
PATH_val = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Training_Set\\Validation\\'
PATH_te = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Testing_Set\\'
Model_path = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\Coronary_Calcium\\Models\\'
model_vers = 'First_Attempt'
input_shape = (1,512,512)


In [5]:
''' Loading Data Sets '''
logging.info("loading datasets")

start_time = time.time()
tr_dict = create_ID_dict(PATH_tr)
val_dict = create_ID_dict(PATH_val)
te_dict = create_ID_dict(PATH_te)
end_time = time.time()

print("Dictionary Creation Took {} seconds".format(end_time-start_time))

start_time = time.time()
tr_data = image_generator(tr_dict,mask = True)
val_data = image_generator(val_dict, mask = True)
te_data = image_generator(te_dict, mask = False)
end_time = time.time()


print("Generators Creation Took {} sec".format(end_time-start_time))


Dictionary Creation Took 0.025994539260864258 seconds
Generators Creation Took 0.0 sec


In [6]:
''' Creating NN'''


if os.path.isdir(Model_path+model_vers)==True:
    logging.info("Loading Network")
    model=tfk.models.load_model("Model_path+model_vers")
    
else:
    logging.info("Creating Network")
#https://towardsdatascience.com/step-by-step-implementation-3d-convolutional-neural-network-in-keras-12efbdd7b130    
    model= tfk.Sequential()
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape, data_format="channels_first", padding='SAME'))
    #model.add(layers.MaxPooling2D(pool_size=(3, 3)))
    #model.add(layers.BatchNormalization(center=True, scale=True))
    #model.add(layers.Dropout(0.5))
    #model.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform', padding ='SAME'))
    #model.add(layers.MaxPooling2D(pool_size=(3, 3)))
    #model.add(layers.BatchNormalization(center=True, scale=True))
    #model.add(layers.Dropout(0.5))
    #model.add(layers.Flatten())
    #model.add(layers.Dense(100, activation='relu', kernel_initializer='he_uniform'))
    #model.add(layers.Dense(50, activation='relu', kernel_initializer='he_uniform'))
    #model.add(layers.Dense(10, activation='relu'))


'''Hyper Parameters'''
logging.info("Adding Hyperparameters")

optimizer=tfk.optimizers.Adadelta() #'Adam' 
loss= "categorical_crossentropy" #'sparse_categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer = optimizer, loss= loss, metrics= metrics )


In [7]:
test_image, test_mask = next(tr_data)
model.predict(test_image)

Image processing
Mask Processing


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-7-4e9f20ab5861>:2) ]] [Op:__inference_predict_function_134]

Function call stack:
predict_function


In [ ]:
model.summary()

In [ ]:
'''n=0
while n<25:
    for img_slice, mask_slice in tr_data:
        print("Image shape is: ",img_slice.shape)
        print("Mask shape is :", mask_slice.shape)
        n+=1'''

In [ ]:
"""Training NN"""
logging.info("Training Neural Network")
#tf.autograph.experimental.do_not_convert(func=model.fit(tr_data,epochs = 10,verbose=10))
model.fit(tr_data,epochs = 10,verbose=10)
#model.save("Model_path+model_vers")

